### Import Dependencies

In [1]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

2024-07-30 07:12:12.600969: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 07:12:12.665098: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 07:12:12.681323: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-30 07:12:12.713482: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-30 07:12:15.293698: W tensorflow/compiler/tf2

### Load EfficientNetB0 pre-trained on ImageNet, excluding the top layers

In [2]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

2024-07-30 07:12:17.985014: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


### Add custom layers on top of the base model

In [3]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Optional additional dense layer
predictions = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification

### Create the full model

In [4]:
model = Model(inputs=base_model.input, outputs=predictions)

### Freeze the base model layers for transfer learning

In [5]:
for layer in base_model.layers:
    layer.trainable = False

### Compile the model

In [6]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
model

<Functional name=functional, built=True>

### Example of loading and preprocessing an image

In [8]:
def load_and_preprocess_image(img_path):
    # Load the image with target size (224, 224) as EfficientNetB0 expects this size

    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Preprocess the image as required by EfficientNet
    return img_array

### Example of making a prediction

In [9]:
image_path = "dataset/single_prediction/cat_or_dog_6.png"
test_image = load_and_preprocess_image(image_path)
result = model.predict(test_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


### Interpreting the result

In [10]:
# Adjusting the threshold
threshold = 0.5  # You can experiment with different values
if result[0][0] > threshold:
    print("Dog")
else:
    print("Cat")

Dog


### Evaluate the model on a dataset

In [11]:
from sklearn.metrics import classification_report, confusion_matrix

def evaluate_model(model, test_generator):
    # Predict the results for the test set
    test_images, test_labels = next(test_generator)  # Or loop through your test data
    predictions = model.predict(test_images)
    predicted_classes = (predictions > 0.5).astype("int32")

    # Print the classification report
    print(classification_report(test_labels, predicted_classes))
    
    # Print confusion matrix
    print(confusion_matrix(test_labels, predicted_classes))





In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the test data generator
test_datagen = ImageDataGenerator(rescale=1./255)  # Rescale pixel values to [0, 1]

# Directory where test images are stored
test_image_directory = "dataset/test_set/"

# Create a test generator
test_generator = test_datagen.flow_from_directory(
    test_image_directory,
    target_size=(224, 224),  # The size expected by EfficientNet
    batch_size=32,
    class_mode='binary',  # Use 'binary' for binary classification
    shuffle=False  # Set to False to ensure that the data is not shuffled
)


Found 2000 images belonging to 2 classes.


### Evaluate the model

In [14]:
loss, accuracy = model.evaluate(test_generator, verbose=1)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


/home/agbor/anaconda3/envs/tf_gpu/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 199s 3s/step - accuracy: 0.1655 - loss: 0.7018
Test Loss: 0.6932
Test Accuracy: 0.5000
